In [1]:
!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv

In [3]:
from views.index import index_blueprint

In [4]:
from flask import Flask, request, g, session, redirect, url_for
from flask import render_template_string, jsonify
from flask_github import GitHub

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [5]:
load_dotenv()

GITHUB_CLIENT_ID = os.getenv('GITHUB_CLIENT_ID')
GITHUB_CLIENT_SECRET = os.getenv('GITHUB_CLIENT_SECRET')

In [6]:
DATABASE_URI = 'sqlite:////srv/jupyterhub/notebooks/github-flask.db'
SECRET_KEY = 'development key'
DEBUG = True


In [7]:
app = Flask(__name__)
#app.config.from_object(__name__)
app.register_blueprint(index_blueprint)

In [8]:
# Set these values
#GITHUB_CLIENT_ID = 'XXX'
#GITHUB_CLIENT_SECRET = 'YYY'

In [9]:
# setup github-flask
# github = GitHub(app)

In [10]:
app.config['GITHUB_CLIENT_ID'] = GITHUB_CLIENT_ID
app.config['GITHUB_CLIENT_SECRET'] = GITHUB_CLIENT_SECRET
app.config['DATABASE_URI'] = DATABASE_URI
# For GitHub Enterprise
#app.config['GITHUB_BASE_URL'] = 'https://dev3.dmtools.info/home/api/v3/'
app.config['GITHUB_BASE_URL'] = 'https://dgithub.com/api/v3/'
#app.config['GITHUB_AUTH_URL'] = 'https://dev3.dmtools.info/home/login/oauth/'
app.config['GITHUB_AUTH_URL'] = 'https://github.com/login/oauth'

github = GitHub(app)

In [11]:
# setup sqlalchemy
engine = create_engine(app.config['DATABASE_URI'])
db_session = scoped_session(sessionmaker(autocommit=False,
                                         autoflush=False,
                                         bind=engine))

In [12]:
Base = declarative_base()
Base.query = db_session.query_property()

In [13]:
def init_db():
    Base.metadata.create_all(bind=engine)


class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    github_access_token = Column(String(255))
    github_id = Column(Integer)
    github_login = Column(String(255))

    def __init__(self, github_access_token):
        self.github_access_token = github_access_token


@app.before_request
def before_request():
    g.user = None
    if 'user_id' in session:
        g.user = User.query.get(session['user_id'])


@app.after_request
def after_request(response):
    db_session.remove()
    return response


@app.route('/home/hello')
def index():
    if g.user:
        t = 'Hello! %s <a href="{{ url_for("user") }}">Get user</a> ' \
            '<a href="{{ url_for("repo") }}">Get repo</a> ' \
            '<a href="{{ url_for("logout") }}">Logout</a>'
        t %= g.user.github_login
    else:
        t = 'Hello! <a href="{{ url_for("login") }}">Login</a>'

    return render_template_string(t)


@github.access_token_getter
def token_getter():
    user = g.user
    if user is not None:
        return user.github_access_token


@app.route('/home/github-callback')
@github.authorized_handler
def authorized(access_token):
    next_url = request.args.get('next') or url_for('index')
    if access_token is None:
        return redirect(next_url)

    user = User.query.filter_by(github_access_token=access_token).first()
    if user is None:
        user = User(access_token)
        db_session.add(user)

    user.github_access_token = access_token

    # Not necessary to get these details here
    # but it helps humans to identify users easily.
    g.user = user
    github_user = github.get('/home/user')
    user.github_id = github_user['id']
    user.github_login = github_user['login']

    db_session.commit()

    session['user_id'] = user.id
    return redirect(next_url)


@app.route('/home/login')
def login():
    if session.get('user_id', None) is None:
        return github.authorize()
    else:
        return 'Already logged in'


@app.route('/home/logout')
def logout():
    session.pop('user_id', None)
    return redirect(url_for('index'))


@app.route('/home/user')
def user():
    return jsonify(github.get('/home/user'))


@app.route('/home/repo')
def repo():
    return jsonify(github.get('/repos/cenkalti/github-flask'))



In [14]:
init_db()

In [ ]:
app.run(host="0.0.0.0", port=5060)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5060
 * Running on http://172.19.0.2:5060 (Press CTRL+C to quit)
172.19.0.4 - - [19/May/2022 14:07:36] "GET /home/hello HTTP/1.1" 200 -
172.19.0.4 - - [19/May/2022 14:07:39] "GET /home/login HTTP/1.1" 302 -
